'1': 필요한 라이브러리 import 하고 dataset이 있는 path 잡아주고 CT폴더 경로 및 RTst로부터 얻은 Contour파일 load함.

In [3]:
#import os
#for i in range(20):
#    try:
#        os.makedirs('D:\junggeyon\CT_aligned_png/Patient%d' %(i+1))
#    except:
#        pass

In [4]:
import os
import numpy as np
import pydicom
from numpngw import write_png

#from PIL import Image #PIL은 8bit image만 다룰수 있음.
#https://stackoverflow.com/questions/62739851/convert-rgb-arrays-to-pil-image
#CT폴더 불러오기
#Contour파일 불러오기
basepath = 'D:\junggeyon\Patient_data/'
dir_list = os.listdir(basepath)
for patient in dir_list:
    if '001' in patient:
        sub_dir_list = os.listdir(basepath + patient)
        for ct in sub_dir_list:
            if 'CT_aligned' == ct:
                sub_sub_dir_list = os.listdir(basepath + patient +'/' + ct)
                for destination in sub_sub_dir_list:
                    CT_folder_path = basepath + patient + '/' + ct +'/'+destination
                    break
        break
    
def PNGandNPYsave(CT_png_vox, CT_voxel):
    np.save(r'D:\junggeyon\Patient_data\001_5978364\CT_aligned\CT_aligned_voxel_001', CT_voxel)
    #np.save('D:\junggeyon\Patient_data\Patient20\patient20CT_png_voxel', CT_png_vox)
    for k in range(len(CT_png_vox)):
        write_png(r'D:\junggeyon\Patient_data\001_5978364\CT_aligned\001ctaligned%d.png' %(k+1), CT_voxel[k])

'2': CT정보를 불러와서 Instance Number에 맞추어 정렬(머리부터 시작)

In [5]:
def GetCTinfo(CT_folder_path):
    CT_dcm_files = []
    fileidx = []
    for top, dir, file in os.walk(CT_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            temp_ct_slice = pydicom.dcmread(CT_folder_path + '/' + filename)
            CT_dcm_files.append(temp_ct_slice)
            idxnum = temp_ct_slice.InstanceNumber
            fileidx.append(idxnum)
        break

    fileidx = np.array(fileidx)
    fileorder = np.argsort(fileidx)

    sorted_CT_file = []
    
    for i in range(len(fileidx)):
        sorted_CT_file.append(CT_dcm_files[fileorder[i]])
        
    return sorted_CT_file, len(fileidx)
    
sorted_CT_file, zsize = GetCTinfo(CT_folder_path)
print(len(sorted_CT_file))

50


'3': 각 slice 마다 Windowing 해줄 함수.

In [6]:
def GetWindowMaxandMin(ct_dicomdata):

    if type(ct_dicomdata.WindowCenter) == pydicom.valuerep.DSfloat:
        window_center = ct_dicomdata.WindowCenter
        window_width = ct_dicomdata.WindowWidth
    else:
        window_center = ct_dicomdata.WindowCenter[0]
        window_width = ct_dicomdata.WindowWidth[0]

    img_max = window_center + window_width // 2
    img_min = window_center - window_width // 2
    
    return img_max, img_min
    

'4': 변환용 Slope와 Intercept구해줄 함수.

In [7]:
def GetSlopeandIntercept(ct_dicomdata):
    slope = float(ct_dicomdata.RescaleSlope)
    intercept = float(ct_dicomdata.RescaleIntercept)
    return slope, intercept

'5': Gray에서 RGB로 변환하기 위해 2차원 배열을 3개 쌓음(ex: 255 ->[255, 255, 255])

In [8]:
def GraytoRGB(arr_slice):
    pixel_rgb = np.stack((arr_slice, )*3, axis = -1)
    return pixel_rgb

'6': 위의 두 함수를 이용하여 Window Center, Window Width와 기존 픽셀값을 HU unit으로 Rescale해주는 값을 얻어 slice array마다 Window를 적용해줌. 그후 16bit에 맞추어 값을 array값을 변환후 RGB화 시켜줌, 변환시 overflow 방지를 위해 int32로 선언
(최댓값이 65535까지 가야하므로.)

In [9]:
def MakeCTvoxel(sorted_CT_file, zsize):
    row = sorted_CT_file[0].Rows
    col = sorted_CT_file[0].Columns
    print(row, col)
    
    CT_voxel = np.zeros((zsize, row, col, 3), 'int32')
    CT_png_vox = np.zeros((zsize, row, col, 3), 'int32')
    for i in range(len(sorted_CT_file)):
        ct_dicomdata = sorted_CT_file[i]
        convert_coeff = 2**(ct_dicomdata.BitsAllocated-1)
        slope, intercept = GetSlopeandIntercept(ct_dicomdata)
        img_max, img_min = GetWindowMaxandMin(ct_dicomdata)
        
        temp_arr_slice = slope * ct_dicomdata.pixel_array + intercept
        temp_arr_slice = temp_arr_slice.astype('int32')
        CT_voxel[i] = GraytoRGB(temp_arr_slice)

        temp_arr_slice[temp_arr_slice < img_min] = img_min
        temp_arr_slice[temp_arr_slice > img_max] = img_max
        
        temp_arr_slice *= round(convert_coeff/(img_max-img_min))
        CT_png_vox[i] = GraytoRGB(temp_arr_slice)

    return CT_voxel, CT_png_vox

CT_voxel, CT_png_vox = MakeCTvoxel(sorted_CT_file, zsize)


512 512


'7': HU unit은 음의 부호도 존재하나 나의 코드에서 png로 저장하기 위해서는 uint16으로 저장해야함. 따라서 위에서 int32으로 수정했던 부분을 uint16으로 저장. 또한 max값을 모두 픽셀의 최대치로, min값을 최소치로 바꿔줌.

In [10]:
#틀렸음..
def RefineCTvoxel(sorted_CT_file, CT_voxel, CT_png_vox):
    hu_add_pixel = 2**11
    hu_div_pixel = 2**12
    convert_coeff = 2**(sorted_CT_file[0].BitsAllocated-1)
    

    CT_voxel += int(hu_add_pixel)
    CT_voxel *= round(2*convert_coeff/hu_div_pixel)
    CT_voxel = CT_voxel.astype('uint16')

    CT_png_vox += int(convert_coeff) - abs(np.min(CT_png_vox))
    CT_png_vox[CT_png_vox <= np.min(CT_png_vox)] = 0
    CT_png_vox[CT_png_vox >= np.max(CT_png_vox)] = 2*convert_coeff - 1
    CT_png_vox = CT_png_vox.astype('uint16')

    
    return CT_voxel, CT_png_vox

CT_voxel, CT_png_vox = RefineCTvoxel(sorted_CT_file, CT_voxel, CT_png_vox)

In [11]:
PNGandNPYsave(CT_png_vox, CT_voxel)

'9': 사진 및 voxel 저장부분.